In [2]:
import respy as rp
import pandas as pd

import string

In [3]:
params, options, data = rp.get_example_model("kw_94_two")
params


value  \
category            name                                  
delta               delta                        0.9500   
wage_a              constant                     9.2100   
                    exp_edu                      0.0400   
                    exp_a                        0.0330   
                    exp_a_square                -0.0005   
                    exp_b                        0.0000   
                    exp_b_square                 0.0000   
wage_b              constant                     8.2000   
                    exp_edu                      0.0800   
                    exp_b                        0.0670   
                    exp_b_square                -0.0010   
                    exp_a                        0.0220   
                    exp_a_square                -0.0005   
nonpec_edu          constant                  5000.0000   
                    at_least_twelve_exp_edu  -5000.0000   
                    not_edu_last_period     -15000.0000   
nonpec_home         constant                 14500.0000   
shocks_sdcorr       sd_a                         0.4000   
                    sd_b                         0.5000   
                    sd_edu                    6000.0000   
                    sd_home                   6000.0000   
                    corr_b_a                     0.0000   
                    corr_edu_a                   0.0000   
                    corr_edu_b                   0.0000   
                    corr_home_a                  0.0000   
                    corr_home_b                  0.0000   
                    corr_home_edu                0.0000   
lagged_choice_1_edu probability                  1.0000   
initial_exp_edu_10  probability                  1.0000   
maximum_exp         edu                         20.0000   

                                                                                       comment  
category            name                                                                        
delta               delta                                                      discount factor  
wage_a              constant                                               log of rental price  
                    exp_edu                          return to an additional year of schooling  
                    exp_a                                     return to same sector experience  
                    exp_a_square                   return to same sector, quadratic experience  
                    exp_b                                    return to other sector experience  
                    exp_b_square                  return to other sector, quadratic experience  
wage_b              constant                                               log of rental price  
                    exp_edu                          return to an additional year of schooling  
                    exp_b                                     return to same sector experience  
                    exp_b_square                   return to same sector, quadratic experience  
                    exp_a                                    return to other sector experience  
                    exp_a_square                  return to other sector, quadratic experience  
nonpec_edu          constant                            constant reward for choosing education  
                    at_least_twelve_exp_edu        reward for going to college (tuition, etc.)  
                    not_edu_last_period                        reward for going back to school  
nonpec_home         constant                         constant reward of non-market alternative  
shocks_sdcorr       sd_a                     Element 1,1 of standard-deviation/correlation ...  
                    sd_b                     Element 2,2 of standard-deviation/correlation ...  
                    sd_edu                   Element 3,3 of standard-deviation/correlation ...  
                    sd_home

We first delete all other occupations.


In [4]:
# TODO:nonpecs need to be dealt with ..

NUM_OCCUPATIONS = 3

assert NUM_OCCUPATIONS > 0
letters = string.ascii_lowercase[:NUM_OCCUPATIONS]

# We first delete all other occupations.
params_debug = params.copy()
occ_base = params_debug.loc["wage_a", :].copy()

params_debug.drop("shocks_sdcorr", level="category", inplace=True)
params_debug.drop("wage_b", level="category", inplace=True)
params_debug.drop("wage_a", level="category", inplace=True)

# We now create a generic new occupation
occ_base = pd.concat([occ_base], keys=['wage_a'], names=['category'])

# We need to drop all experience terms that refer to other occupations
for label in occ_base.index.get_level_values(level="name"):
    if "exp_" in label and "edu" not in label:
        occ_base.drop(label, level="name", inplace=True)

# We now create new experience variables.
for letter in letters:
    for ext_ in ["", "_square"]:
        name = f"exp_{letter}" + ext_
        info = {"category": ["wage_a"], "name": [name], "value": [0.0], "comment": ["comment"]}
        
        info = pd.DataFrame.from_dict(info).set_index(["category", "name"])
        occ_base = occ_base.append(info)    

# We now add it to the params dataframe
for letter in letters:   
    occ_params = occ_base.loc["wage_a", :].copy()
    occ_params = pd.concat([occ_params], keys=[f'wage_{letter}'], names=['category'])
    
    params_debug = params_debug.append(occ_params)
    
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(params_debug["value"])

category             name                   
delta                delta                          0.95
nonpec_edu           constant                    5000.00
                     at_least_twelve_exp_edu    -5000.00
                     not_edu_last_period       -15000.00
nonpec_home          constant                   14500.00
lagged_choice_1_edu  probability                    1.00
initial_exp_edu_10   probability                    1.00
maximum_exp          edu                           20.00
wage_a               constant                       9.21
                     exp_edu                        0.04
                     exp_a                          0.00
                     exp_a_square                   0.00
                     exp_b                          0.00
                     exp_b_square                   0.00
                     exp_c                          0.00
                     exp_c_square                   0.00
wage_b               constant              

Now need to construct te covariance_matrix

In [5]:
non_occupation = ["edu", "home"]
occupation = list(letters)

from itertools import product

In [6]:
names = list()
labels = non_occupation + occupation

for labels in product(labels, labels):
    left, right = labels
     
    # We do not need correlation 
    if left == right:
        names += [f"sd_{left}"]
        continue
        
    # We check if 
    if f"corr_{right}_{left}" in names:
        continue
        
    names += [f"corr_{left}_{right}"]

In [7]:
# TODO. NEEDS omre flexible
if True:
    
    index = [
    'sd_a', 'sd_b', 'sd_c', 'sd_edu', 'sd_home', 
    'corr_b_a', 'corr_c_a', 'corr_c_b', 
    'corr_edu_a', 'corr_edu_b', 'corr_edu_c', 
    'corr_home_a', 'corr_home_b', 'corr_home_c', 'corr_home_edu']    


    indices = list()
    for name in index:
        indices += [("shocks_sdcorr", name)]

        
        
index = pd.MultiIndex.from_tuples(indices, names=["category", "name"])
shocks_sdcorr = pd.DataFrame(index=index, columns=["value", "comment"])

In [8]:
    
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(shocks_sdcorr["value"])

category       name         
shocks_sdcorr  sd_a             NaN
               sd_b             NaN
               sd_c             NaN
               sd_edu           NaN
               sd_home          NaN
               corr_b_a         NaN
               corr_c_a         NaN
               corr_c_b         NaN
               corr_edu_a       NaN
               corr_edu_b       NaN
               corr_edu_c       NaN
               corr_home_a      NaN
               corr_home_b      NaN
               corr_home_c      NaN
               corr_home_edu    NaN
Name: value, dtype: object


In [9]:
for category, name in shocks_sdcorr.index:
    if "sd_" in name:
        shocks_sdcorr.loc[(category, name), "value"] = 1.0
    elif "corr_" in name:
        shocks_sdcorr.loc[(category, name), "value"] = 0.0
    else:
        raise AssertionError

    
params_debug = params_debug.append(shocks_sdcorr)

    
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(params_debug["value"])

category             name                   
delta                delta                       0.95
nonpec_edu           constant                    5000
                     at_least_twelve_exp_edu    -5000
                     not_edu_last_period       -15000
nonpec_home          constant                   14500
lagged_choice_1_edu  probability                    1
initial_exp_edu_10   probability                    1
maximum_exp          edu                           20
wage_a               constant                    9.21
                     exp_edu                     0.04
                     exp_a                          0
                     exp_a_square                   0
                     exp_b                          0
                     exp_b_square                   0
                     exp_c                          0
                     exp_c_square                   0
wage_b               constant                    9.21
                     exp_edu         

In [10]:
params_annica = pd.read_pickle("params_annica.pkl")

In [11]:
options["core_state_space_filters"] = ["period > 0 and exp_{choices_w_exp} == period and lagged_choice_1 != '{choices_w_exp}'",
 "period > 0 and exp_a + exp_b + exp_c + exp_edu == period and lagged_choice_1 == '{choices_wo_exp}'",
 "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
 "lagged_choice_1 == '{choices_w_wage}' and exp_{choices_w_wage} == 0",
 "period == 0 and lagged_choice_1 == '{choices_w_wage}'"]

options["n_periods"] = 10
options["covariates"]["exp_c_square"] = "exp_c ** 2"

In [18]:
annica_index = params_annica.index
params_debug =params_debug.reindex(annica_index)
params_debug["value"] = params_debug["value"].astype("float")

In [19]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(params_debug["value"])

category             name                   
delta                delta                          0.95
initial_exp_edu_10   probability                    1.00
lagged_choice_1_edu  probability                    1.00
maximum_exp          edu                           20.00
nonpec_edu           at_least_twelve_exp_edu    -5000.00
                     constant                    5000.00
                     not_edu_last_period       -15000.00
nonpec_home          constant                   14500.00
wage_a               constant                       9.21
                     exp_a                          0.00
                     exp_a_square                   0.00
                     exp_b                          0.00
                     exp_b_square                   0.00
                     exp_c                          0.00
                     exp_c_square                   0.00
                     exp_edu                        0.04
wage_b               constant              

In [20]:
simulate = rp.get_simulate_func(params_debug, options)


In [23]:
df = simulate(params_debug)

In [31]:
df.sum().sum()

21461750223.46615

In [33]:
import numpy as np
np.testing.assert_equal(df["Choice"].nunique(), 5)
np.testing.assert_almost_equal(df.sum().sum(), 21461750223.46615)